In [1]:
import pandas as pd
import numpy as np
import math
import TreeNode

Class DecisionTree with all helper functions

In [2]:
class DecisionTree:
    
    #preprocess data
    def preprocessData(self,filename):
        data=pd.read_csv(filename, sep=" ", header=None)
        data.columns=['index','occupied','prices','music','location','VIP','favorite beer','Enjoy']
        data=data[1:]
        data=data.drop('index',axis=1)
        data['occupied']=data['occupied'].str.replace(',','')
        data['prices']=data['prices'].str.replace(',','')
        data['music']=data['music'].str.replace(',','')
        data['location']=data['location'].str.replace(',','')
        data['VIP']=data['VIP'].str.replace(',','')
        data['favorite beer']=data['favorite beer'].str.replace(',','')
        data['Enjoy']=data['Enjoy'].str.replace(';','')
        #shuffle data
        #data=data.sample(frac=1).reset_index(drop=True)
        training_data=data[0:17]
        training_label=data.iloc[0:17,6]
        attributes=list(data.columns)
        target=attributes[-1]
        training_data=training_data.values.tolist()
        testingData=data.ix[17:21,0:6]
        test_labels=data.ix[17:21,6]
        testingData=testingData.values.tolist()
        return training_data, testingData, attributes, target, test_labels, data, training_label
    
    #calculate entropy
    def entropy(self,attributes, data, targetAttr):
        valFreq = {}
        dataEntropy = 0.0
        #find index of the target attribute
        i = 0
        i=attributes.index(targetAttr)
    
        # Calculate the frequency of each of the values in the target attr
        for entry in data:
            if entry[i] in valFreq:
                valFreq[entry[i]] += 1.0
            else:
                valFreq[entry[i]]  = 1.0

        # Calculate the entropy of the data for the target attr
        for freq in valFreq.values():
            dataEntropy += (-freq/len(data)) * math.log(freq/len(data), 2) 
        
        return dataEntropy
    
    #calculate gain
    def gain(self,attributes, data, attr, targetAttr):
      
        valFreq = {}
        subsetEntropy = 0.0
    
        #find index of the attribute
        i = attributes.index(attr)

        # Calculate the frequency of each of the values in the target attribute
        for entry in data:
            if entry[i] in valFreq:
                valFreq[entry[i]] += 1.0
            else:
                valFreq[entry[i]]  = 1.0
        # Calculate the sum of the entropy for each subset of records weighted
        # by their probability of occuring in the training set.
        for val in valFreq.keys():
            valProb        = valFreq[val] / sum(valFreq.values())
            dataSubset     = [entry for entry in data if entry[i] == val]
            subsetEntropy += valProb * self.entropy(attributes, dataSubset, targetAttr)

        # Subtract the entropy of the chosen attribute from the entropy of the
        # whole data set with respect to the target attribute (and return it)
        return (self.entropy(attributes, data, targetAttr) - subsetEntropy)
    
    #return attribute with max gain
    def chooseAttr(self,data, attributes, target):
        best = attributes[0]
        maxGain = 0;
        priority=['occupied', 'prices','music','location','VIP','favorite beer']
        for attr in attributes:
            if attr!='Enjoy':
                newGain = self.gain(attributes, data, attr, target) 
                if newGain==maxGain:
                    if priority.index(attr)>priority.index(best):
                        best=attr
                        maxGain=newGain
                    
                elif newGain>maxGain:
                    maxGain = newGain
                    best = attr
        return best

    #get values in the column of the given attribute 
    def getSubClasses(self,data, attributes, attr):
        index = attributes.index(attr)
        values = []
        for entry in data:
            if entry[index] not in values:
                values.append(entry[index])
        return values

    #return rows corresponding to a sub class of an attribute
    def getRowsCorrespondingToSubClassOfAttribute(self,data, attributes, best, val):
        examples = [[]]
        index = attributes.index(best)
        for entry in data:
            #find entries with the give value
            if (entry[index] == val):
                newEntry = []
                #add value if it is not in best column
                for i in range(0,len(entry)):
                    if(i != index):
                        newEntry.append(entry[i])
                examples.append(newEntry)
        examples.remove([])
        return examples

    #return attribute whose occurence is max
    def majority(self,data,attributes,target):
        valFreq={}
        index=attributes.index(target)

        for v in data:
            if v[index] in valFreq:
                valFreq[v[index]]+=1
            else:
                valFreq[v[index]]=1
        maxAttr=0
        major="" 
        for key in valFreq.keys():
            if valFreq[key]==maxAttr:
                major='Tie'
             
            if valFreq[key]>maxAttr:
                major=key
                maxAttr=valFreq[key]
                
        return major

    #generate decision tree recursively
    def generateTree(self,data, attributes, target, recursion):
        recursion+=1
        data=data[:]
        vals=[record[attributes.index(target)] for record in data]
        default=self.majority(data, attributes, target)
        if not data or len(attributes)-1 <=0:
            return default
        elif vals.count(vals[0])==len(vals):
            return vals[0]
        else:
            best=self.chooseAttr(data, attributes, target)
            tree= {best:{}}
        
            for val in self.getSubClasses(data, attributes, best):
                examples=self.getRowsCorrespondingToSubClassOfAttribute(data,attributes, best, val)
                newattr=attributes[:]
                newattr.remove(best)
                subtree=self.generateTree(examples, newattr, target, recursion)
                tree[best][val]=subtree
            
        return tree

    #predict output of test data
    def predictOutput(self,testdata, attributes):
        count=0
        res=[]
        for entry in testdata:
            count+=1
            tempD=dict_.copy()
            result=""
            while isinstance(tempD, dict):
                root=TreeNode.TreeNode(list(tempD.keys())[0], tempD[list(tempD.keys())[0]])
                tempD=tempD.get(list(tempD.keys())[0])
                index=attributes.index(str(root.value))
                value=entry[index]
                if value in tempD.keys():
                    child=TreeNode.TreeNode(value, tempD[value])
                    result=tempD[value]
                    tempD=tempD[value]
                else:
                    print('No data')
                    result="Tie"
                    break
            
            res.append(result)
        return res    

Create object and call DecisionTree()

In [3]:
obj=DecisionTree()
training_data, testing_data, attributes, target, test_labels, data, training_label=obj.preprocessData("dt-data.txt")

Print testing data, labels

In [4]:
testing_data

[['High', 'Cheap', 'Loud', 'City-Center', 'No', 'Yes'],
 ['Low', 'Normal', 'Quiet', 'City-Center', 'No', 'No'],
 ['Low', 'Expensive', 'Loud', 'Mahane-Yehuda', 'No', 'No'],
 ['Moderate', 'Normal', 'Quiet', 'Talpiot', 'No', 'No'],
 ['Low', 'Normal', 'Quiet', 'City-Center', 'No', 'No']]

In [5]:
training_label


1      No
2     Yes
3     Yes
4      No
5     Yes
6     Yes
7      No
8     Yes
9     Yes
10     No
11     No
12     No
13    Yes
14    Yes
15    Yes
16     No
17    Yes
Name: Enjoy, dtype: object

In [6]:
test_labels

17    Yes
18     No
19     No
20    Yes
21    Yes
Name: Enjoy, dtype: object

In [7]:
dict_=obj.generateTree(training_data,attributes, target,0)

Print tree in json format

In [8]:
dict_

{'location': {'City-Center': {'occupied': {'High': 'Yes',
    'Low': 'No',
    'Moderate': 'Yes'}},
  'Ein-Karem': {'occupied': {'Low': 'No', 'Moderate': 'Yes'}},
  'German-Colony': {'favorite beer': {'No': 'No', 'Yes': 'Yes'}},
  'Mahane-Yehuda': 'Yes',
  'Talpiot': 'No'}}

Accuracy

In [9]:
sum(test_labels==obj.predictOutput(testing_data,[a for a in attributes if a!='Enjoy']))/len(test_labels)*100

40.0

Accuracy on training set, fits too perfectly? Clear case of overfitting

In [12]:
res=obj.predictOutput(training_data,[a for a in attributes if a!='Enjoy'])
sum(training_label==res)/len(training_label)*100

100.0

Predict outcome of ['Moderate','Cheap','Loud','City-Center','No','No']

In [13]:
obj.predictOutput([['Moderate','Cheap','Loud','City-Center','No','No']],attributes)

['Yes']